# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [ ]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [ ]:
ruta = "/content/dataset_banco_clean.csv"
data = pd.read_csv(ruta)
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45184,51,technician,married,tertiary,no,825.0,no,no,cellular,17,nov,977.0,3,-1.0,0,unknown,yes
45185,71,retired,divorced,primary,no,1729.0,no,no,cellular,17,nov,456.0,2,-1.0,0,unknown,yes
45186,72,retired,married,secondary,no,5715.0,no,no,cellular,17,nov,1127.0,5,184.0,3,success,yes
45187,57,blue-collar,married,secondary,no,668.0,no,no,telephone,17,nov,508.0,4,-1.0,0,unknown,no


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

# 1. Procesamiento


Al utilizar un dataset ya limpiado previamente, podremos pasar directamente a su codificación.

In [ ]:
# Mostrar las primeras filas del conjunto de datos
print(data.head())

# Información del conjunto de datos
print(data.info())

# Estadísticas descriptivas
print(data.describe())


   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no   2143.0     yes   no   
1   44    technician   single  secondary      no     29.0     yes   no   
2   33  entrepreneur  married  secondary      no      2.0     yes  yes   
3   47   blue-collar  married    unknown      no   1506.0     yes   no   
4   33       unknown   single    unknown      no      1.0      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may     261.0         1   -1.0         0  unknown  no  
1  unknown    5   may     151.0         1   -1.0         0  unknown  no  
2  unknown    5   may      76.0         1   -1.0         0  unknown  no  
3  unknown    5   may      92.0         1   -1.0         0  unknown  no  
4  unknown    5   may     198.0         1   -1.0         0  unknown  no  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45189 entries, 0 to 45188
Data columns (total 17 columns):
 #

En primer lugar, codificamos las variables categóricas ordinalmente.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Inicializar el codificador para variables categóricas
label_encoder = LabelEncoder()

# Lista de variables categóricas a codificar
categorical_variables = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Iterar sobre las variables categóricas y codificarlas ordinalmente
for variable in categorical_variables:
    data[variable + '_encoded'] = label_encoder.fit_transform(data[variable])

# Codificar 'y' a valores numéricos (1 para 'yes', 0 para 'no')
data['y_encoded'] = label_encoder.fit_transform(data['y'])

# Agregar 'y_encoded' a la lista de características numéricas
numerical_features = ['y_encoded'] + data.select_dtypes(include=['int', 'float']).columns.tolist()

# Eliminar las columnas originales de las variables categóricas si lo deseas
data.drop(categorical_variables + ['y'], axis=1, inplace=True)

# Verificar los valores únicos de las variables codificadas
for variable in categorical_variables:
    print(variable + '_encoded:', data[variable + '_encoded'].unique())

# Verificar los valores únicos de 'y_encoded'
print('y_encoded:', data['y_encoded'].unique())


job_encoded: [ 4  9  2  1 11  5  0  7  6 10  3  8]
marital_encoded: [1 2 0]
education_encoded: [2 1 3 0]
default_encoded: [0 1]
housing_encoded: [1 0]
loan_encoded: [0 1]
contact_encoded: [2 0 1]
month_encoded: [ 8  9  6  5  1 10  2  4  3  7  0 11]
poutcome_encoded: [3 0 1 2]
y_encoded: [0 1]


In [ ]:
data

,age,balance,day,duration,campaign,pdays,previous,job_encoded,marital_encoded,education_encoded,default_encoded,housing_encoded,loan_encoded,contact_encoded,month_encoded,poutcome_encoded,y_encoded
0,58,2143.0,5,261.0,1,-1.0,0,4,1,2,0,1,0,2,8,3,0
1,44,29.0,5,151.0,1,-1.0,0,9,2,1,0,1,0,2,8,3,0
2,33,2.0,5,76.0,1,-1.0,0,2,1,1,0,1,1,2,8,3,0
3,47,1506.0,5,92.0,1,-1.0,0,1,1,3,0,1,0,2,8,3,0
4,33,1.0,5,198.0,1,-1.0,0,11,2,3,0,0,0,2,8,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45184,51,825.0,17,977.0,3,-1.0,0,9,1,2,0,0,0,0,9,3,1
45185,71,1729.0,17,456.0,2,-1.0,0,5,0,0,0,0,0,0,9,3,1
45186,72,5715.0,17,1127.0,5,184.0,3,5,1,1,0,0,0,0,9,2,1
45187,57,668.0,17,508.0,4,-1.0,0,1,1,1,0,0,0,1,9,3,0


Para terminar, realizamos un escalamiento de los datos, ya que vamos a probar todos los algoritmos y será una buena manera de compararlos.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Inicializar el escalador
scaler = StandardScaler()

# Escalar las características numéricas
data_scaled = data.copy()  # Hacemos una copia del conjunto de datos
data_scaled[numerical_features] = scaler.fit_transform(data[numerical_features])

# Verificar los valores únicos de las características escaladas (si lo deseas)
for feature in numerical_features:
    print("Valores únicos de", feature, "después del escalado:", data_scaled[feature].unique())


Valores únicos de y_encoded después del escalado: [-0.363927   2.7478038]
Valores únicos de age después del escalado: [ 1.60698229  0.28851426 -0.7474249   0.57104313 -0.55907232 -1.21830634
  0.10016169  0.19433798  0.0059854  -1.12413005  1.512806    0.94774828
  0.38269055  1.79533486  1.41862971 -0.84160119 -1.5008352  -0.08819089
 -0.18236717  1.04192456  0.47686684 -0.46489604  0.7593957   1.70115858
  1.13610085 -0.37071975  0.85357199  1.23027714  1.32445343  0.66521941
 -1.59501149 -0.27654346 -0.93577747 -1.02995376 -1.31248262 -0.65324861
 -1.68918777 -1.40665891  1.88951115 -1.78336406 -1.87754035 -1.97171664
  2.36039259  1.98368744  3.96138948  3.20797918  2.45456888  2.73709774
  2.2662163   2.54874516  2.17204001  2.64292145  2.92545031  2.83127403
 -2.06589292  3.30215546  4.14974205  2.07786373  4.62062349  3.86721319
  3.0196266   3.11380289  3.49050804  3.67886061  4.99732864  3.58468433
  3.39633175  4.24391834  5.09150493  3.7730369  -2.16006921  4.5264472
  4.055

Ahora tenemos el dataset original en el campod data, y el escalado por si se necesitara en un futuro en el campo data_scaled.

# 2. Generar split de los datos

Dividimos el conjunto de datos en conjuntos de entrenamiento y prueba para poder evaluar el desempeño del modelo.

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en características (X) y la variable objetivo (y)
X = data_scaled.drop('y_encoded', axis=1)  # Características
y = data_scaled['y_encoded']  # Variable objetivo

# Dividir los datos en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos de entrenamiento y prueba
print("Forma del conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Forma del conjunto de prueba:", X_test.shape, y_test.shape)


Forma del conjunto de entrenamiento: (36151, 16) (36151,)
Forma del conjunto de prueba: (9038, 16) (9038,)


# 3. Evaluación de modelos

Entrenamos y evaluamos varios modelos de clasificación utilizando el conjunto de datos de entrenamiento. Probaremos árboles de decisión, kNN y SVM vistos en el notebook anterior.

In [ ]:
# Convertir las etiquetas de destino a enteros para que evitar errores
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Inicializar el clasificador de Árboles de Decisión
dt_classifier = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo de Árboles de Decisión
dt_classifier.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_dt = dt_classifier.predict(X_test)

# Calcular la precisión del modelo
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Precisión del modelo de Árboles de Decisión:", accuracy_dt)


Precisión del modelo de Árboles de Decisión: 0.8744191192741757


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Inicializar el clasificador kNN
knn_classifier = KNeighborsClassifier()

# Entrenar el modelo kNN
knn_classifier.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_knn = knn_classifier.predict(X_test)

# Calcular la precisión del modelo
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("Precisión del modelo kNN:", accuracy_knn)


Precisión del modelo kNN: 0.8954414693516265


In [ ]:
from sklearn.svm import SVC

# Inicializar el clasificador SVM
svm_classifier = SVC(random_state=42)

# Entrenar el modelo SVM
svm_classifier.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_svm = svm_classifier.predict(X_test)

# Calcular la precisión del modelo
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Precisión del modelo SVM:", accuracy_svm)


Precisión del modelo SVM: 0.8997565833148927


Observamos que de los 3 modelos, para este dataset el algoritmo SVC es el más óptimo , que obtiene una precisión cercana al 90%.

# 4. Ensemble

A continuación, vamos a utilizar un ensemble de modelos para mejorar la precisión de nuestras predicciones. Un ensemble combina las predicciones de varios modelos base para obtener una predicción final más precisa.

Vamos a utilizar la técnica de Bagging, que es una de las técnicas de ensemble más comunes. Bagging consiste en entrenar varios modelos base (por lo general, árboles de decisión) en diferentes subconjuntos del conjunto de datos de entrenamiento y luego promediar sus predicciones para obtener la predicción final.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Inicializar el modelo base (Árbol de decisión)
base_model = DecisionTreeClassifier()

# Inicializar el modelo Bagging
bagging_model = BaggingClassifier(base_model, n_estimators=10, random_state=42)

# Entrenar el modelo Bagging
bagging_model.fit(X_train, y_train)

# Calcular la precisión del modelo Bagging en el conjunto de prueba
accuracy_bagging = bagging_model.score(X_test, y_test)

print("Precisión del modelo Bagging:", accuracy_bagging)


Precisión del modelo Bagging: 0.9018588183226377


Vemos que gracias al Bagging superamos la precisión de los algoritmos por separado, superado el 90%. Vamos a probar un par de algoritmos más.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Inicializar el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo Random Forest
rf_model.fit(X_train, y_train)

# Calcular la precisión del modelo Random Forest en el conjunto de prueba
accuracy_rf = rf_model.score(X_test, y_test)

print("Precisión del modelo Random Forest:", accuracy_rf)


Precisión del modelo Random Forest: 0.9080548793980969


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Inicializar el modelo Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo Gradient Boosting
gb_model.fit(X_train, y_train)

# Calcular la precisión del modelo Gradient Boosting en el conjunto de prueba
accuracy_gb = gb_model.score(X_test, y_test)

print("Precisión del modelo Gradient Boosting:", accuracy_gb)


Precisión del modelo Gradient Boosting: 0.908276167293649


Tras esto, observamos que el ensemble más preciso para nuestro dataset de los probados es el Gradient Boosting, llegando al 91% de precisión.

# 5. Métricas



Calculamos métricas de evaluación como precisión, recall, F1-score, matriz de confusión, etc., para evaluar el desempeño de los modelos en el conjunto de prueba.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Definir una función para calcular y mostrar métricas de evaluación
def evaluate_model(model, X_test, y_test):
    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular y mostrar la matriz de confusión
    print("Matriz de confusión:")
    print(confusion_matrix(y_test, y_pred))

    # Calcular y mostrar el reporte de clasificación
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))

# Evaluar Árbol de Decisión
print("Métricas para Árbol de Decisión:")
evaluate_model(dt_classifier, X_test, y_test)

# Evaluar kNN
print("\nMétricas para kNN:")
evaluate_model(knn_classifier, X_test, y_test)

# Evaluar SVM
print("\nMétricas para SVM:")
evaluate_model(svm_classifier, X_test, y_test)

# Evaluar Bagging
print("\nMétricas para Bagging:")
evaluate_model(bagging_model, X_test, y_test)

# Evaluar Random Forest
print("\nMétricas para Random Forest:")
evaluate_model(rf_model, X_test, y_test)

# Evaluar Gradient Boosting
print("\nMétricas para Gradient Boosting:")
evaluate_model(gb_model, X_test, y_test)


Métricas para Árbol de Decisión:
Matriz de confusión:
[[7395  583]
 [ 552  508]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      7978
           2       0.47      0.48      0.47      1060

    accuracy                           0.87      9038
   macro avg       0.70      0.70      0.70      9038
weighted avg       0.88      0.87      0.88      9038


Métricas para kNN:
Matriz de confusión:
[[7763  215]
 [ 730  330]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      7978
           2       0.61      0.31      0.41      1060

    accuracy                           0.90      9038
   macro avg       0.76      0.64      0.68      9038
weighted avg       0.88      0.90      0.88      9038


Métricas para SVM:
Matriz de confusión:
[[7847  131]
 [ 775  285]]

Reporte de clasificación:
              precision    recall  f1-score   su

Vemos que el **árbol de decisión** tiene una alta precisión y recall para la clase 0, pero una precisión y recall más bajos para la clase 2. Esto puede indicar un desequilibrio en el conjunto de datos o un sesgo del modelo hacia la clase mayoritaria.

El modelo **kNN** tiene una buena precisión para la clase 0 pero un recall bajo para la clase 2. Esto indica que el modelo está clasificando correctamente la mayoría de las instancias de la clase 0, pero tiene dificultades para identificar correctamente las instancias de la clase 2.

Por otro lado, vemos que tanto **Random Forest** como **Gradient Boosting** muestran un rendimiento equilibrado en términos de precisión y recall para ambas clases, lo que demuestra que son la mejor opción para nuestro estudio.

# Conclusiones

Como conclusión podemos observar que todos los modelos evaluados muestran un rendimiento decente en términos de precisión global, pero también se identifican diferencias en cómo manejan las diferentes clases y características del conjunto de datos.

Algunos modelos han demostrado ser más robustos y equilibrados en su capacidad para predecir correctamente ambas clases, mientras que otros mostraron desequilibrios o sesgos hacia una clase en particular.

